In [ ]:
%pip install ipywidgets==8.1.1

Zeiss Dimension C-mount machine vision lenses: https://www.zeiss.com/content/dam/consumer-products/downloads/industrial-lenses/brochures/en/brochure-zeiss-industrial-lenses.pdf

Schneider-Kreuznach Tourmaline C-mount lenses: https://schneiderkreuznach.com/application/files/7616/8492/7579/Schneider_Kreuznach_Products_2023.pdf#page=4

In [ ]:
from ipywidgets import interactive,IntSlider,FloatSlider # import interact for interactive figures https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
import ipywidgets as widgets
import numpy as np # import array manipulation and maths library numpy under the alias np
import astropy.units as u # import our astropy.units submodule under the alias u for real-world unit
# from IPython.display import display
from IPython.display import Latex, display

In [ ]:
print("\033[1mDiffraction Limited Angular & Spatial Resolution\033[0m")

#print explanation and equation
print("\nUsing the Raleigh criterion (with sine small angle approximation):")
display(Latex(rf"$$\theta = 1.22 \frac{{\lambda}}{{D}}$$"))

print("\nthen calculating the spatial resolution from the angular resolution and target range:")

#print explanation and equation
display(Latex(rf"$$\Delta\ell = 2 r \tan \left( \frac{{\theta}}{2} \right)$$"))

#update function
def update(wavelength_nm, focal_length_mm, relative_aperture, target_range_km):
    #wavelength description
    if 10 <= wavelength_nm and wavelength_nm < 122:
        wavelength_description = "Extreme Ultraviolet"
    elif 122 <= wavelength_nm and wavelength_nm < 400:
        wavelength_description = "Ultraviolet"
    elif 400 <= wavelength_nm and wavelength_nm < 475:
        wavelength_description = "Visible Band [Blue]"
    elif 475 <= wavelength_nm and wavelength_nm < 575:
        wavelength_description = "Visible Band [Green]"
    elif 575 <= wavelength_nm and wavelength_nm < 751:
        wavelength_description = "Visible Band [Red]"
    elif 751 <= wavelength_nm and wavelength_nm < 1400:
        wavelength_description = "Near-Infrared"
    elif 1400 <= wavelength_nm and wavelength_nm < 3000:
        wavelength_description = "Short-Wavelength Infrared"
    elif 3000 <= wavelength_nm and wavelength_nm < 8000:
        wavelength_description = "Mid-Wavelength Infrared"
    elif 8000 <= wavelength_nm and wavelength_nm <= 15000:
        wavelength_description = "Long-Wavelength Infrared"
    else:
        wavelength_description = "spectrum band unknown"
    
    wavelength = wavelength_nm * u.nm #add units
    print("\nλ = {:.0f} ({})".format(wavelength, wavelength_description))
    
    focal_length = focal_length_mm * u.mm #add units
    print("f = {:.0f}".format(focal_length))
    
    print("F/ = {:.0f} (relative aperture)".format(relative_aperture))
    
    aperture_diameter = focal_length/relative_aperture
    print("D = {:.1f} (absolute aperture diameter)".format(aperture_diameter))
    
    #calculate the diffraction limited angular resulution using the Raliegh criterian
    #and small angle approximation for sine
    dl_angular_resolution = np.arcsin(1.22 * wavelength/aperture_diameter).to(u.arcsec)
    
    print("\nDiffraction limited angular resolution: θ = {:.2f}".format(dl_angular_resolution))
    
    target_range = target_range_km * u.km #add units
    print("\nr = {:.0f}".format(target_range))
    
    #calculate the diffraction limited angular resulution using the Raliegh criterian
    #and small angle approximation for sine
    spatial_resolution = (2*target_range*np.tan(dl_angular_resolution/2)).to(u.m)
    
    print("\nDiffraction Limited Spatial Resolution: Δℓ = {:.2f}".format(spatial_resolution))
    
    return wavelength, wavelength_description, aperture_diameter, dl_angular_resolution, target_range, spatial_resolution

layout = widgets.Layout(width="600px") #make the widgets wider so the slider bars are bigger

#create slider and link to update function
optical_system = interactive(update,
    wavelength_nm=IntSlider(
        min=200,
        max=15000,
        step=50,
        value=750,
        description="Wavelenth (λ) in nm:",
        style={"description_width" : "initial"},
        continuous_update = True,
        layout = layout),
    focal_length_mm=IntSlider(
         min=8,
         max=800,
         step=1,
         value=50,
         description="Focal Length (f) in mm:",
         style={"description_width" : "initial"},
         continuous_update = True,
         layout = layout),
    relative_aperture=FloatSlider(
         min=1,
         max=8,
         step=0.1,
         readout_format=".1f",
         value=2,
         description="Relative Aperture F/):",
         style={"description_width" : "initial"},
         continuous_update = True,
         layout = layout),
    target_range_km=IntSlider(
        min=1,
        max=200,
        step=1,
        value=10,
        description="Range to Target in km:",
        style={"description_width" : "initial"},
        continuous_update = True,
        layout = layout),
);

display(optical_system)

wavelength, wavelength_description, aperture_diameter, dl_angular_resolution, target_range, spatial_resolution = optical_system.result